In [1]:
import sys
sys.path.append('/home/jovyan/modules')
from pre import *
from connectors import *
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 1000)
from statsmodels.stats.power import tt_ind_solve_power 
sys.path.append('/home/jovyan/AB/Content methodology') 
from econometrics.general import *
!pip install tabulate
warnings.filterwarnings("ignore")

В одном из RTE-чатиков СЕО сокрушался по поводу того, что нет полки с избранными товарами.
Гипотеза кажется сомнительной (число избранных товаров и в гросери, и в рте, очевидно, никакущее), но интересной к проработке.

Что необходимо

Проверить избранные блюда в контексте RTE: 

* проникновение избранных блюд в рте к активным RTE-пользователям за последний год
* как много избранных блюд еще живы - являются доступными к покупке в ресторанах
* какое-нибудь хитрое сравнение про избранные в рте и гросери (поштормим вместе с аналитиком), чтобы понять паттерны избранных в рте и гросери

Если увидим, что это может быть интересным - запилим полку с избранными товарами и проведем АБ

In [2]:
# Выберем все sku из RTE, которые были активны за последний год
# Имена меняются, так что берем просто самую последнюю версию

date_start = '2022-12-06'
 
##  city,
rte_active_sku = \
read_sql_query(f'''
select sku
, argMax(product_name, li_created_at)  as product_name_
, groupUniqArray(r.name) as ret_names


from line_items
left join int_spree_retailers r on line_items.retailer_id = r.id

where li_created_at>toDate('{date_start}')
and master_category like '%%RTE%%'
and r.name not in ('Test Много Лосося','OperationsTEST')
group by sku
''')


In [3]:
# Теперь выберем тех, кто заказывал в RTE за последний год
date_start = '2022-12-06'

rte_active_users = \
read_sql_query(f'''
select user_id from line_items
where li_created_at>toDate('{date_start}')
and master_category like '%%RTE%%'
group by user_id
''')



In [4]:
# И посмотрим на их избранное за последний год - среднее кол-во товаров RTE в избранном, 

In [5]:
q = f'''
select user_id, sku from
"pg-prod-paas-content-favorites".public.favorites 
where created_at > date('{date_start}')
'''
favorites_df = read_sql_query(q, con = 'trino')

In [6]:
favorites_df_cut = \
favorites_df[favorites_df['user_id'].isin(rte_active_users.user_id)]

In [7]:
favorites_df_cut = \
favorites_df_cut[favorites_df['sku'].isin(rte_active_sku.sku.astype(int))]

In [8]:
#Кто добавляет товары - сколько их ?
favorites_df_cut.groupby('user_id').count().mean()

sku    2.58602
dtype: float64

In [34]:
favorites_df_cut.groupby('user_id').count().describe()

,sku
count,62061.000000
mean,2.586020
std,5.415144
min,1.000000
25%,1.000000
50%,1.000000
75%,3.000000
max,445.000000


In [9]:
favorites_df_cut.user_id.nunique()

62061

In [10]:
rte_active_users.shape

(1117498, 1)

In [11]:
favorites_df_cut.user_id.nunique() / rte_active_users.shape[0]

0.0555356698624964

In [12]:
# Из 1104533 пользователей только у 61425 есть избранное в RTE - около 6%

In [14]:
# Выберем все sku из RTE, которые были активны за последний месяц
date_start = '2023-11-06'

rte_active_sku_1mnth = \
read_sql_query(f'''
select sku from line_items
where li_created_at>toDate('{date_start}')
and master_category like '%%RTE%%'
group by sku
''')

In [15]:
favorites_df_cut_1mnth = \
favorites_df[favorites_df['user_id'].isin(rte_active_users.user_id)]

In [16]:
favorites_df_cut_1mnth = \
favorites_df_cut_1mnth[favorites_df_cut_1mnth['sku'].isin(rte_active_sku_1mnth.sku.astype(int))]

In [17]:
favorites_df_cut_1mnth.shape[0]/favorites_df_cut.shape[0]

0.9027982877544535

# Гипотеза 0 - что вообще добавляют в избранное? Сделать разбивку по ресторанам и пользвоателя

In [18]:
rte_active_sku.sku = \
rte_active_sku.sku.astype(int)

In [19]:
favorites_df_upd = \
favorites_df_cut.merge(rte_active_sku)

In [20]:
favorites_df_upd['ret_names_len'] = favorites_df_upd.ret_names.map(len)

In [21]:
favorites_df_upd['ret_names'] = favorites_df_upd.ret_names.map(set)

In [22]:
favorites_df_upd['ret_names_s'] = favorites_df_upd.ret_names.map(str)

In [23]:
fav_df_cnt = \
favorites_df_upd.groupby(['sku','product_name_','ret_names_s']).count().reset_index()\
[['sku','product_name_','ret_names_s','user_id']].sort_values(by = 'user_id',ascending=False).reset_index(drop=True)

In [24]:
fav_df_cnt.user_id.describe()

count    53457.000000
mean         3.002245
std         13.178461
min          1.000000
25%          1.000000
50%          1.000000
75%          2.000000
max        741.000000
Name: user_id, dtype: float64

In [32]:
fav_df_cnt.shape

(53457, 4)

In [ ]:
fav_df_cnt.groupby('ret_names_s').count().mean()
## Наполненность 

In [25]:
np.quantile(fav_df_cnt.user_id,0.99)

38.0

In [31]:
fav_df_cnt_huge.shape

(532, 4)

In [30]:
fav_df_cnt_huge = \
fav_df_cnt[fav_df_cnt.user_id > 38]
fav_df_cnt_huge.head(20)

,sku,product_name_,ret_names_s,user_id
0,24591923,Кинг Фри большой,{'БУРГЕР КИНГ'},741
1,24591903,Чизбургер,{'БУРГЕР КИНГ'},722
2,25344057,Баскет Фри,{'KFC'},603
3,24591922,Кинг Фри станд,{'БУРГЕР КИНГ'},497
4,24591914,Чикенбургер,{'БУРГЕР КИНГ'},471
5,25344145,Баскет L 24 Острых Крылышка,{'KFC'},463
6,25344024,Соус Сырный,{'KFC'},454
7,24591843,Воппер,{'БУРГЕР КИНГ'},451
8,26449888,Биг Хит,{'Вкусно - и точка'},365
9,26449782,Чикенбургер,{'Вкусно - и точка'},362


In [28]:
fav_df_cnt_huge.groupby('ret_names_s').count()[['sku']].sort_values(by = 'sku', ascending=False).reset_index()

,ret_names_s,sku
0,{'БУРГЕР КИНГ'},157
1,{'KFC'},103
2,{'Вкусно - и точка'},63
3,{'ТАНУКИ'},53
4,"{""Rostic's""}",41
5,{'Суши Мастер'},22
6,{'МНОГО ЛОСОСЯ'},22
7,{'Суши Маркет'},13
8,{'Якитория'},10
9,{'Best Price Pizza'},7


## Часть 2. Полку делаем, сколько пользователей ее увидят?

In [ ]:
# [минимум 2 позиции в самом любимом ритейлере], сколько на ритейлере?? Проверить чувтствительность. На цифрах ответить ок не ок 
# Так как юзер добавил в избранное, срезаем ему шаги и повышаем конверсию
# Наша ЦА - RTE добавляют - 5%, больше 2х - 5/4% -> RTE от всех - 0.25% от всего сервиса будут видеть полку на ~1 рте ритейлере
## а если разбить по ритейлерам?

In [45]:
favorites_df_upd.groupby(['ret_names_s','user_id']).count()['sku'].describe()

count    77418.000000
mean         2.073045
std          2.590361
min          1.000000
25%          1.000000
50%          1.000000
75%          2.000000
max         87.000000
Name: sku, dtype: float64

In [49]:
favorites_df_upd.groupby(['ret_names_s','user_id']).count()[['sku']].reset_index().query('sku>=2').shape

(26509, 3)

In [110]:
rte_fav_users = \
favorites_df_upd.groupby(['ret_names_s','user_id']).count()[['sku']].reset_index().query('sku>=2').user_id.to_list()

In [114]:
len(rte_fav_users)

26509

In [112]:
# куда 5/26к делись?
map_users = \
read_sql_query(f'''
    select uuid, id as user_id from int_spree_users
    where id in {tuple(rte_fav_users)}

    ''')
# Кто это?
 

In [ ]:
rte_fav_users

In [115]:
rte_fav_users_uuid =  map_users.uuid.to_list()

In [101]:
platforms = [ "android",'web']
end_date = dt.datetime(2023, 12, 9).date() 
lengths = [1,2,3]
start_date = end_date - dt.timedelta(days = 7*max(lengths))

In [121]:
for el in pd.date_range(start = start_date, end = end_date):
    exp_date = el.strftime('%Y-%m-%d')
    print(exp_date)
    
    try:
        a = pd.read_feather(f'discount/{exp_date}.feather')
    except:
        to_exp_data = \
    read_sql_query(f'''
    select product_added
    , conversion
    , gmv_per_user
    , orders
    , checkout_loaded
    , cart_viewed
    , anonymous_id
    , date_msk
    , uuid
    , platform
    from cdm.ab__metrics_data
    where date_msk = toDate('{exp_date}')
    and uuid<>''

    ''')
        to_exp_data = to_exp_data[to_exp_data['uuid'].isin(rte_fav_users_uuid)].reset_index(drop=True)

        to_exp_data.to_feather(f'discount/{exp_date}.feather')
    

2023-11-18
2023-11-19
2023-11-20
2023-11-21
2023-11-22
2023-11-23
2023-11-24
2023-11-25
2023-11-26
2023-11-27
2023-11-28
2023-11-29
2023-11-30
2023-12-01
2023-12-02
2023-12-03
2023-12-04
2023-12-05
2023-12-06
2023-12-07
2023-12-08
2023-12-09


In [122]:
platforms = [ "android",'desktop']
traffic_proportions = [ 1]
thresholds = [0.05, 0.01]
num_of_group = 2

In [123]:
metrics = [('product_added_more_than_one','to_enum'),
          ('conversion_first','to_enum'),
          ('checkout_loaded','cart_viewed'),
          ('conversion_second','to_enum')]

types = ['ratio','ratio','ratio','ratio','ratio']

In [124]:
%%time
res = {"platform": [], 
       "length": [], 
       "traffic_proportion": [], 
       "alpha":[], 
       "est":[],
       "sd":[],
       "mde_abs": [], 
       "mde_percent": [],
       "metric": [],
      'nobs_per_group':[]}

for l in tqdm(lengths):
    new_start_date = end_date - dt.timedelta(days = 7*l)
    new_df_list = []
    for d in pd.date_range(start = new_start_date, end = end_date):
        exp_date = d.strftime('%Y-%m-%d')
        new_df_list.append(pd.read_feather(f'discount/{exp_date}.feather'))
    
    new_df = pd.concat(new_df_list)
    
    for platform in tqdm(platforms):
        
        new_df_platform = new_df[new_df['platform']==platform]
        new_df_platform = new_df_platform.groupby(['platform','anonymous_id']).sum().reset_index()
        
        new_df_platform['product_added_more_than_one'] = new_df_platform['product_added']>1 
        new_df_platform.replace({'product_added_more_than_one' : {True:1, False:0}}, inplace=True)

        new_df_platform['conversion_first'] = new_df_platform['conversion']>0
        new_df_platform.replace({'conversion_first' : {True:1, False:0}}, inplace=True)

        new_df_platform['conversion_second'] = new_df_platform['conversion']>1
        new_df_platform.replace({'conversion_second' : {True:1, False:0}}, inplace=True)
        
        new_df_platform['to_enum'] = 1
        
        for proportion in traffic_proportions:
            df = new_df_platform.sample(frac = proportion/num_of_group)
            for alpha in thresholds:
                
                for idx, m in enumerate(metrics):
                    
                    mde_data = calculate_metrics_mde_data(df, m, alpha, platform, num_of_group*proportion, metric_type=types[idx])
                    
                    res["platform"].append(platform)
                    res["length"].append(l)
                    res["traffic_proportion"].append(proportion)
                    res["alpha"].append(alpha)
                    res["est"].append(mde_data[0])
                    res["sd"].append(mde_data[1])
                    res['nobs_per_group'].append(mde_data[2])
                    res["mde_percent"].append(mde_data[3])
                    res["mde_abs"].append(mde_data[3] * mde_data[0])
                    res["metric"].append(m)
        

 33%|███▎      | 1/3 [00:00<00:00,  3.90it/s]

4866
4866
4866
4866
4866
4866
4866
4866
4866
4866
366
366
366
366
366
366
366
366
366
366



 67%|██████▋   | 2/3 [00:00<00:00,  3.47it/s]

5528
5528
5528
5528
5528
5528
5528
5528
5528
5528
528
528
528
528
528
528
528
528
528
528



100%|██████████| 3/3 [00:00<00:00,  3.20it/s]

5890
5890
5890
5890
5890
5890
5890
5890
5890
5890
649
649
649
649
649
649
649
649
649
649
CPU times: user 975 ms, sys: 44.2 ms, total: 1.02 s
Wall time: 940 ms


In [125]:
res_df = pd.DataFrame(res)

In [126]:
metric_name_dict = {('product_added_more_than_one', 'to_enum'):["Конверсия из захода в добавление товара",'target'],
                   ('conversion_first', 'to_enum'):["Конверсия из захода в оформление заказа","target"],
                   ('conversion_second', 'to_enum'):["Конверсия из захода в повторный заказ","info"]
                   }

In [127]:
names=[]
types=[]
for i in res_df.iterrows():
    names.append(metric_name_dict[i[1].metric][0])
    types.append(metric_name_dict[i[1].metric][1])

res_df['name'] = names
res_df['type'] = types 

In [128]:
res_df = res_df.query('(type=="target" and alpha==0.05) or (type=="info" and alpha==0.01)').reset_index(drop=True)

In [136]:
beauty_res_df = res_df.copy()

In [137]:
prc_metrics = ['Конверсия из захода в добавление товара','Конверсия из захода в оформление заказа'\
               ,'Конверсия из корзины в чекаут','Конверсия из захода в повторный заказ']

In [138]:
beauty_res_df['est'] = beauty_res_df.apply(\
                lambda x: str(int(x.est*100))+"%" if x.name in prc_metrics else round(x.est,2) , axis = 1)

In [139]:
beauty_res_df['mde_percent'] = beauty_res_df.apply(\
                lambda x: str(round(x.mde_percent*100,1))+"%"  , axis = 1)

In [140]:
beauty_res_df.shape

(30, 12)

In [144]:
# target
print(
    beauty_res_df[beauty_res_df['type']=='target']\
    [beauty_res_df['platform']=='android']\
[['length','platform','name','traffic_proportion','est','sd','nobs_per_group','mde_percent']]\
    [beauty_res_df['name'].isin(['Конверсия из захода в добавление товара','Конверсия из захода в оформление заказа'])]
    .sort_values(by = ['length','platform','traffic_proportion','name'])
    .to_markdown(index=False)
)

|   length | platform   | name                                    |   traffic_proportion |   est |       sd |   nobs_per_group | mde_percent   |
|---------:|:-----------|:----------------------------------------|---------------------:|------:|---------:|-----------------:|:--------------|
|        1 | android    | Конверсия из захода в добавление товара |                    1 |  0.65 | 0.477462 |             4866 | 4.2%          |
|        1 | android    | Конверсия из захода в оформление заказа |                    1 |  0.57 | 0.49482  |             4866 | 4.9%          |
|        2 | android    | Конверсия из захода в добавление товара |                    1 |  0.72 | 0.448698 |             5528 | 3.3%          |
|        2 | android    | Конверсия из захода в оформление заказа |                    1 |  0.67 | 0.471724 |             5528 | 3.8%          |
|        3 | android    | Конверсия из захода в добавление товара |                    1 |  0.75 | 0.432213 |             5890 | 3

In [148]:
beauty_res_df[beauty_res_df['type']=='target']\
[beauty_res_df['platform']=='android']\
[['length','platform','name','traffic_proportion','est','sd','nobs_per_group','mde_percent']]\
[beauty_res_df['name'].isin(['Конверсия из захода в добавление товара','Конверсия из захода в оформление заказа'])]\
.sort_values(by = ['length','platform','traffic_proportion','name']).reset_index(drop=True)

,length,platform,name,traffic_proportion,est,sd,nobs_per_group,mde_percent
0,1,android,Конверсия из захода в добавление товара,1,0.65,0.477462,4866,4.2%
1,1,android,Конверсия из захода в оформление заказа,1,0.57,0.494820,4866,4.9%
2,2,android,Конверсия из захода в добавление товара,1,0.72,0.448698,5528,3.3%
3,2,android,Конверсия из захода в оформление заказа,1,0.67,0.471724,5528,3.8%
4,3,android,Конверсия из захода в добавление товара,1,0.75,0.432213,5890,3.0%
5,3,android,Конверсия из захода в оформление заказа,1,0.70,0.460126,5890,3.4%


# Гипотеза 1 - в избранное добавляют то, что часто недоступно

In [29]:
favorites_df_cut

,user_id,sku
11551,19072885,21972843
20650,9681446,22941002
29723,11888247,26253298
30172,19038649,23233521
30173,19038649,23233519
...,...,...
20350573,17840394,25344145
20350575,17840394,25344024
20350576,17840394,25344164
20350578,17840394,25344175


# Гипотеза 2 - в избранное добавляют то, что часто заказывали и удобно составлять корзину из избранного